 <img src="http://news.nationalgeographic.com/content/dam/news/2015/12/15/pandas/01pandainsemination.adapt.768.1.jpg" class="pull-right" width=300>

## Math 157: Intro to Mathematical Software
## UC San Diego, winter 2018

## February 9, 2018: Pandas

Administrivia:

- As usual, virtual office hours run 3-7pm, and the homework is due at 8pm.
- For homework problem 4c, please take $k=1,2,3,4$ instead of $k=1,2,3,4,5$ to avoid crashing your Jupyter kernel. (If you are having trouble even for $k=4$, try restarting your project. If that doesn't help, you might need to make your code more memory-efficient.)
- Advance warning: next week, my office hours will take place Thursday 3-4 instead of 4-5 due to a schedule conflict.


### What is pandas?

> "[pandas](http://pandas.pydata.org/) is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language."

It is used all over the world (including at the UCSD Guardian, I am told) as a primary tool for transferring data sets into Python for the purposes of manipulation, visualization, etc.

pandas is a member of the [SciPy ecosystem](https://www.scipy.org/).

### Warning: big data = big headaches (sometimes)

When using pandas, it is easy to create **very** large files in your course project. The disk quota for your project is currently about 300MB; overrunning this may cause unexpected problems (e.g., getting your project stuck in a "locked" state). Contact course staff if this happens, but more importantly do not leave large files in your project any longer than necessary!


### 10 minutes to pandas

The following is adapted from [this tutorial](https://pandas.pydata.org/pandas-docs/stable/10min.html), with a few changes to handle differences between Sage and pure Python. The "10 minutes" is misleading; we will spend the whole hour on this and not get through it all. It would take a few hours to go through everything carefully, which I would recommend if you plan to use this extensively.


In [1]:
import pandas as pd
import numpy as np

One fundamental data structure in pandas is a `Series`, which is similar to a list.

In [2]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0      1
1      3
2      5
3    NaN
4      6
5      8
dtype: object

It is possible to specify alternate labels instead of the default 0,1,...; more on this later.

Another fundamental data structure is a `DataFrame`, which is basically a list of `Series`. A good metaphor for how a `DataFrame` behaves is an Excel spreadsheet; in fact, it is not hard to import and export Excel spreadsheets using this data structure.

In [3]:
dates = pd.date_range('20130101', periods=6) # oops, this gives an error
dates

TypeError: periods must be a number, got 6

Let me take a quick aside to illustrate why this happened. Remember that Sage feeds commands through a preparser before delivering them to Python; that's how you are able to type `3^4` in Sage to mean exponentiation instead of `3**4`. But you can also call the preparser yourself to see how it behaves. 

In [5]:
preparse("3^4")

'Integer(3)**Integer(4)'

In [4]:
preparse("dates = pd.date_range('20130101', periods=6)") # Let's see why this happened.

"dates = pd.date_range('20130101', periods=Integer(6))"

So Sage is making sure that integer literals get created as Sage integers by default, rather than Python integers. pandas doesn't know how to fix this, because it doesn't know anything about Sage; but we can fix this manually.

In [6]:
dates = pd.date_range('20130101', periods=int(6))
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

Better yet, we can turn off the preparser so that we don't have to keep doing this. Don't forget to turn it back on when you want to switch back to Sage syntax!

In [7]:
preparser(False)

In [8]:
dates = pd.date_range('20130101', periods=6)
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

## End of aside.

Here we convert a numpy array into a `DataFrame`.

In [9]:
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2013-01-01,-0.810389,0.824075,1.382784,-0.143679
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194
2013-01-03,0.953788,-0.298446,1.345920,0.179292
2013-01-04,0.668588,0.401658,0.002766,1.018606
2013-01-05,-1.510081,0.957084,-1.579342,-0.963909
2013-01-06,-1.319244,-0.351347,-1.126652,-0.473196


Notice that Jupyter knows to how to pretty-print this!

In [10]:
type(df)

<class 'pandas.core.frame.DataFrame'>

In [0]:
df. # Try tab-completion to see what methods are available

Here we construct a `DataFrame` in a different way, from a dictionary of list-like objects.

In [11]:
df2 = pd.DataFrame({ 'A' : 1.,
                     "G" : pd.Timestamp('20130102'),
                     'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                     'D' : np.array([1,2,3,4],dtype='int32'),
                     'E' : pd.Categorical(["test","train","test","train"]),
                     'F' : ['foo', 'bar', 'foo', 'bar'] })

df2

,A,C,D,E,F,G
0,1.0,1.0,1,test,foo,2013-01-02
1,1.0,1.0,2,train,bar,2013-01-02
2,1.0,1.0,3,test,foo,2013-01-02
3,1.0,1.0,4,train,bar,2013-01-02


The data in each column has to be a particular type. However, as a fallback, pandas allows the type `object` for arbitrary Python objects.

In [12]:
df2.dtypes # Like in numpy, this is an attribute rather than a method.

A           float64
C           float32
D             int32
E          category
F            object
G    datetime64[ns]
dtype: object

There are various ways to view data in a `DataFrame`...

In [13]:
df.head(2)

,A,B,C,D
2013-01-01,-0.810389,0.824075,1.382784,-0.143679
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194


In [14]:
df.tail()

,A,B,C,D
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194
2013-01-03,0.953788,-0.298446,1.345920,0.179292
2013-01-04,0.668588,0.401658,0.002766,1.018606
2013-01-05,-1.510081,0.957084,-1.579342,-0.963909
2013-01-06,-1.319244,-0.351347,-1.126652,-0.473196


In [15]:
df.index

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [16]:
df.columns

Index([u'A', u'B', u'C', u'D'], dtype='object')

In [17]:
df.values

array([[-0.81038868,  0.82407517,  1.38278369, -0.1436792 ],
       [-0.50901509, -0.43543885, -1.22481693, -0.24919439],
       [ 0.95378815, -0.29844621,  1.34592039,  0.17929153],
       [ 0.66858785,  0.40165846,  0.00276616,  1.01860646],
       [-1.51008067,  0.9570845 , -1.57934228, -0.96390948],
       [-1.3192444 , -0.35134697, -1.12665158, -0.4731958 ]])

It is also easy to get some quick statistics on the data.

In [18]:
df.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,-0.421059,0.182931,-0.199890,-0.105347
std,1.022698,0.625744,1.322702,0.669163
min,-1.510081,-0.435439,-1.579342,-0.963909
25%,-1.192030,-0.338122,-1.200276,-0.417195
50%,-0.659702,0.051606,-0.561943,-0.196437
75%,0.374187,0.718471,1.010132,0.098549
max,0.953788,0.957084,1.382784,1.018606


In [19]:
df.T # Transpose, like a matrix

,2013-01-01 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-04 00:00:00,2013-01-05 00:00:00,2013-01-06 00:00:00
A,-0.810389,-0.509015,0.953788,0.668588,-1.510081,-1.319244
B,0.824075,-0.435439,-0.298446,0.401658,0.957084,-0.351347
C,1.382784,-1.224817,1.345920,0.002766,-1.579342,-1.126652
D,-0.143679,-0.249194,0.179292,1.018606,-0.963909,-0.473196


In [20]:
df.sort_index(axis=1, ascending=False) # Sort by index

,D,C,B,A
2013-01-01,-0.143679,1.382784,0.824075,-0.810389
2013-01-02,-0.249194,-1.224817,-0.435439,-0.509015
2013-01-03,0.179292,1.345920,-0.298446,0.953788
2013-01-04,1.018606,0.002766,0.401658,0.668588
2013-01-05,-0.963909,-1.579342,0.957084,-1.510081
2013-01-06,-0.473196,-1.126652,-0.351347,-1.319244


In [21]:
df.sort_values(by='B') # Sort by values in a column

,A,B,C,D
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194
2013-01-06,-1.319244,-0.351347,-1.126652,-0.473196
2013-01-03,0.953788,-0.298446,1.345920,0.179292
2013-01-04,0.668588,0.401658,0.002766,1.018606
2013-01-01,-0.810389,0.824075,1.382784,-0.143679
2013-01-05,-1.510081,0.957084,-1.579342,-0.963909


Now let's look at ways to extract (or "select", to continue the Excel metaphor) sections of data.

In [22]:
df['A'] # Select a single column

2013-01-01   -0.810389
2013-01-02   -0.509015
2013-01-03    0.953788
2013-01-04    0.668588
2013-01-05   -1.510081
2013-01-06   -1.319244
Freq: D, Name: A, dtype: float64

In [23]:
df[0:3] # Use slice notation to select a range of rows

,A,B,C,D
2013-01-01,-0.810389,0.824075,1.382784,-0.143679
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194
2013-01-03,0.953788,-0.298446,1.345920,0.179292


In [24]:
df['20130102':'20130104'] # You can also slice by index rather than position.

,A,B,C,D
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194
2013-01-03,0.953788,-0.298446,1.345920,0.179292
2013-01-04,0.668588,0.401658,0.002766,1.018606


In [25]:
df.loc[dates[0]] # Select a single row

A   -0.810389
B    0.824075
C    1.382784
D   -0.143679
Name: 2013-01-01 00:00:00, dtype: float64

In [26]:
df.loc[:,['A','B']] # Select multiple columns

,A,B
2013-01-01,-0.810389,0.824075
2013-01-02,-0.509015,-0.435439
2013-01-03,0.953788,-0.298446
2013-01-04,0.668588,0.401658
2013-01-05,-1.510081,0.957084
2013-01-06,-1.319244,-0.351347


In [27]:
df.loc['20130102':'20130104',['A','B']] # Select rows and columns

,A,B
2013-01-02,-0.509015,-0.435439
2013-01-03,0.953788,-0.298446
2013-01-04,0.668588,0.401658


In [28]:
df.loc['20130102',['A','B']] # Note the reduction of dimension here...

A   -0.509015
B   -0.435439
Name: 2013-01-02 00:00:00, dtype: float64

In [29]:
df.loc[dates[0],'A'] # .. and here.

-0.81038867560804528

In [30]:
df.at[dates[0],'A'] # Another way to select a single entry, may be more efficient in practice.

-0.81038867560804528

In [31]:
df.iloc[3] # Select by position instead of index

A    0.668588
B    0.401658
C    0.002766
D    1.018606
Name: 2013-01-04 00:00:00, dtype: float64

In [32]:
df.iloc[3:5,0:2] # Slice by position, a la python

,A,B
2013-01-04,0.668588,0.401658
2013-01-05,-1.510081,0.957084


In [33]:
df.iloc[[1,2,4],[0,2]] # Again, you can use lists instead of ranges

,A,C
2013-01-02,-0.509015,-1.224817
2013-01-03,0.953788,1.345920
2013-01-05,-1.510081,-1.579342


In [34]:
df.iloc[1:3,:]

,A,B,C,D
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194
2013-01-03,0.953788,-0.298446,1.345920,0.179292


In [35]:
df.iloc[:,1:3]

,B,C
2013-01-01,0.824075,1.382784
2013-01-02,-0.435439,-1.224817
2013-01-03,-0.298446,1.345920
2013-01-04,0.401658,0.002766
2013-01-05,0.957084,-1.579342
2013-01-06,-0.351347,-1.126652


In [36]:
df.iloc[1,1]

-0.43543884541049577

In [37]:
df.iat[1,1] # Equivalent to the previous one, but more efficient

-0.43543884541049577

You can also select rows or columns based on boolean conditions, as in a list comprehension; but the notation here is more compact.

In [38]:
df[df.A > 0]

,A,B,C,D
2013-01-03,0.953788,-0.298446,1.345920,0.179292
2013-01-04,0.668588,0.401658,0.002766,1.018606


In [39]:
df[df > 0] # Pick out all entries in the DataFrame satisfying this condition

,A,B,C,D
2013-01-01,NaN,0.824075,1.382784,NaN
2013-01-02,NaN,NaN,NaN,NaN
2013-01-03,0.953788,NaN,1.345920,0.179292
2013-01-04,0.668588,0.401658,0.002766,1.018606
2013-01-05,NaN,0.957084,NaN,NaN
2013-01-06,NaN,NaN,NaN,NaN


The missing entries here have been set to `np.nan` (NaN stands for "Not a Number"). This is the pandas analogue of having an empty cell in an Excel spreadsheet. Operations generally skip over missing data.

In [40]:
dft = df[df > 0]
dft.describe() # Pay attention to the counts

,A,B,C,D
count,2.000000,3.000000,3.000000,2.000000
mean,0.811188,0.727606,0.910490,0.598949
std,0.201667,0.290007,0.786328,0.593485
min,0.668588,0.401658,0.002766,0.179292
25%,0.739888,0.612867,0.674343,0.389120
50%,0.811188,0.824075,1.345920,0.598949
75%,0.882488,0.890580,1.364352,0.808778
max,0.953788,0.957084,1.382784,1.018606


In [41]:
df2 = df.copy() # Make a copy so we don't mess up the original
df2['E'] = ['one', 'one','two','three','four','three'] # Add a new column
df2

,A,B,C,D,E
2013-01-01,-0.810389,0.824075,1.382784,-0.143679,one
2013-01-02,-0.509015,-0.435439,-1.224817,-0.249194,one
2013-01-03,0.953788,-0.298446,1.345920,0.179292,two
2013-01-04,0.668588,0.401658,0.002766,1.018606,three
2013-01-05,-1.510081,0.957084,-1.579342,-0.963909,four
2013-01-06,-1.319244,-0.351347,-1.126652,-0.473196,three


In [43]:
df2[df2['E'].isin(['two','three'])] # Pick out rows based on whether the entry in a given column belongs to a given list

,A,B,C,D,E
2013-01-03,0.953788,-0.298446,1.345920,0.179292,two
2013-01-04,0.668588,0.401658,0.002766,1.018606,three
2013-01-06,-1.319244,-0.351347,-1.126652,-0.473196,three


You can write values into a single entry, or a range; this is analogous to pasting into an Excel spreadsheet (or in the case of a single value, simply typing in a new value).

In [44]:
df.at[dates[0],'A'] = 0 # Reference by index...

In [45]:
df.iat[0,1] = 0 # ... or position

In [46]:
df.loc[:,'D'] = np.array([5] * len(df)) # Set a whole column

In [47]:
df # See what happened

,A,B,C,D
2013-01-01,0.000000,0.000000,1.382784,5
2013-01-02,-0.509015,-0.435439,-1.224817,5
2013-01-03,0.953788,-0.298446,1.345920,5
2013-01-04,0.668588,0.401658,0.002766,5
2013-01-05,-1.510081,0.957084,-1.579342,5
2013-01-06,-1.319244,-0.351347,-1.126652,5


In [48]:
df2 = df.copy()
df2[df2 > 0] = -df2 # Select entries based on a criterion, then set those
df2 # No positive entries should appear!

,A,B,C,D
2013-01-01,0.000000,0.000000,-1.382784,-5
2013-01-02,-0.509015,-0.435439,-1.224817,-5
2013-01-03,-0.953788,-0.298446,-1.345920,-5
2013-01-04,-0.668588,-0.401658,-0.002766,-5
2013-01-05,-1.510081,-0.957084,-1.579342,-5
2013-01-06,-1.319244,-0.351347,-1.126652,-5


It is possible to change indices in an existing `DataFrame`.

In [49]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])
df1.loc[dates[0]:dates[1],'E'] = 1
df1

,A,B,C,D,E
2013-01-01,0.000000,0.000000,1.382784,5,1.0
2013-01-02,-0.509015,-0.435439,-1.224817,5,1.0
2013-01-03,0.953788,-0.298446,1.345920,5,NaN
2013-01-04,0.668588,0.401658,0.002766,5,NaN


Returning to missing data, one can handle it in various ways.

In [50]:
df1.dropna(how='any') # Drop rows with missing data

,A,B,C,D,E
2013-01-01,0.000000,0.000000,1.382784,5,1.0
2013-01-02,-0.509015,-0.435439,-1.224817,5,1.0


In [51]:
df1.fillna(value=5) # Fill missing data with a default value

,A,B,C,D,E
2013-01-01,0.000000,0.000000,1.382784,5,1.0
2013-01-02,-0.509015,-0.435439,-1.224817,5,1.0
2013-01-03,0.953788,-0.298446,1.345920,5,5.0
2013-01-04,0.668588,0.401658,0.002766,5,5.0


In [52]:
pd.isna(df1) # Create a Boolean array identifying the locations of missing data

,A,B,C,D,E
2013-01-01,False,False,False,False,False
2013-01-02,False,False,False,False,False
2013-01-03,False,False,False,False,True
2013-01-04,False,False,False,False,True


Let me skip a few sections here.

- Binary operations
- Statistics
- Applying functions to the data (like the Python `map` function on a list)
- Histogramming
- String processing 

One can combine data in various ways, such as concatenation...

In [53]:
df = pd.DataFrame(np.random.randn(10, 4))
df

,0,1,2,3
0,1.509890,-1.517178,-0.279475,0.645688
1,-0.250527,0.016750,0.884331,0.642849
2,-0.598965,-0.249450,-0.923939,-0.014415
3,-2.135250,1.892228,0.627839,0.308394
4,0.282967,0.015005,0.322434,0.587913
5,-0.529997,-0.434233,1.334112,0.273121
6,-0.487898,0.564374,-0.687684,-1.266690
7,-0.700254,0.232348,1.190121,-0.664170
8,-1.119840,1.148164,-0.608473,-0.105387
9,-1.059987,-2.396005,0.296617,0.428761


In [54]:
pieces = [df[7:], df[3:7], df[:3]]
pd.concat(pieces) # Reassemble

,0,1,2,3
7,-0.700254,0.232348,1.190121,-0.664170
8,-1.119840,1.148164,-0.608473,-0.105387
9,-1.059987,-2.396005,0.296617,0.428761
3,-2.135250,1.892228,0.627839,0.308394
4,0.282967,0.015005,0.322434,0.587913
5,-0.529997,-0.434233,1.334112,0.273121
6,-0.487898,0.564374,-0.687684,-1.266690
0,1.509890,-1.517178,-0.279475,0.645688
1,-0.250527,0.016750,0.884331,0.642849
2,-0.598965,-0.249450,-0.923939,-0.014415


... or merging, as in a SQL database. (As a mathematician, I like to think of this as a "Cartesian product".)

In [55]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'foo'], 'rval': [4, 5]})

In [56]:
left

,key,lval
0,foo,1
1,foo,2


In [57]:
right

,key,rval
0,foo,4
1,foo,5


In [58]:
pd.merge(left, right, on='key')

,key,lval,rval
0,foo,1,4
1,foo,1,5
2,foo,2,4
3,foo,2,5


In [59]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

In [60]:
left

,key,lval
0,foo,1
1,bar,2


In [61]:
right

,key,rval
0,foo,4
1,bar,5


In [62]:
pd.merge(left, right, on='key')

,key,lval,rval
0,foo,1,4
1,bar,2,5


Let's skip some more.

- Appending rows
- Grouping (this means splitting the data on some criteria, applying a different function to each group, then recombining)
- Reshaping
- Time series (more on these when we focus on statistics)
- Categoricals (values limited to a small number of options, e.g., letter grades)

Plotting uses matplotlib.

In [63]:
import matplotlib.pyplot as plt

In [64]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts = ts.cumsum() # Cumulative sum
ts.plot(); # Don't print this, just do the plot

Plotting a `DataFrame` gives you superimposed plots.

In [65]:
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index,
                        columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best');

Finally, one can move data in and out of CSV files...

In [66]:
df.to_csv('foo.csv')
pd.read_csv('foo.csv')

,Unnamed: 0,A,B,C,D
0,2000-01-01,0.689727,0.639506,0.448357,0.910182
1,2000-01-02,1.801161,0.867688,-0.546138,1.860435
2,2000-01-03,0.854324,0.378218,-0.688508,1.765520
3,2000-01-04,2.052232,1.632131,-1.288768,3.527727
4,2000-01-05,1.091321,1.856794,-1.797786,5.512203
5,2000-01-06,1.447722,2.966903,-2.853800,6.774737
6,2000-01-07,1.268280,3.452749,-4.235693,6.719425
7,2000-01-08,0.125393,5.050571,-3.695521,6.833089
8,2000-01-09,-1.252676,4.821481,-3.640297,6.478155
9,2000-01-10,0.087111,4.132111,-2.555681,5.956343


... or Excel spreadsheets.

In [67]:
df.to_excel('foo.xlsx', sheet_name='Sheet1')
pd.read_excel('foo.xlsx', 'Sheet1', index_col=None, na_values=['NA'])

,A,B,C,D
2000-01-01,0.689727,0.639506,0.448357,0.910182
2000-01-02,1.801161,0.867688,-0.546138,1.860435
2000-01-03,0.854324,0.378218,-0.688508,1.765520
2000-01-04,2.052232,1.632131,-1.288768,3.527727
2000-01-05,1.091321,1.856794,-1.797786,5.512203
2000-01-06,1.447722,2.966903,-2.853800,6.774737
2000-01-07,1.268280,3.452749,-4.235693,6.719425
2000-01-08,0.125393,5.050571,-3.695521,6.833089
2000-01-09,-1.252676,4.821481,-3.640297,6.478155
2000-01-10,0.087111,4.132111,-2.555681,5.956343


After running the previous example, try switching to the file view. You can try downloading the new files and trying to open them in a spreadsheet program (Excel, OpenOffice, etc.).